
# Aho-Corasick Algorithm for Pattern Matching
The Aho-Corasick algorithm is a classical search algorithm that builds an Aho-Corasick trie (or finite state machine) and efficiently searches for multiple patterns in a text. It combines two key features:

## Trie Construction: We build a tree structure of all the keywords.
## Failure Links: Failure links ensure that when a match fails at a particular node, we can efficiently "fall back" to a previous state that might match the input.

We’ll begin by defining the key structures and then implement the Aho-Corasick algorithm.

### Step 1: Define the State Case Class
In the Aho-Corasick algorithm, each state represents a node in the trie. A state has:
- ID: Unique identifier for the state.
- Successors: A map of input characters leading to the next state.
- EndState: A boolean flag to mark if this state corresponds to the end of a keyword.
- Keyword: An optional field to store the keyword if this is an end state.

In [1]:
import scala.collection.mutable.Queue
case class State(ID: Int, Successor: Map[String, Int], endState: Boolean, keyword: Option[String] = None)


import scala.collection.mutable.Queue

defined class State

### Step 2: Build the Trie on basis of the keywords
The heart of the Aho-Corasick algorithm is the Trie, which is the key to be able to run a Finite State Machine over the inputs. We need to build a path for each keyword respecting the links between them aswell as the endstates and inputs

In [2]:

def buildGraph(keywords: List[String]): Map[Int, State] = {
  var nextID = 0
  var states = Map[Int, State](nextID -> State(nextID, Map(), endState = false)) // Root state

  for (keyword <- keywords) {
    var currentStateID = 0 // Start at the root state

    for (char <- keyword) {
      val currentState = states(currentStateID)

      // Check if there's already a state for this character, else create a new state
      val nextStateID = currentState.Successor.getOrElse(char.toString, {
        nextID += 1
        nextID
      })

      // Update the current state to include the new successor
      states = states.updated(currentStateID, currentState.copy(Successor = currentState.Successor + (char.toString -> nextStateID)))

      // Add the new state if it doesn't exist
      if (!states.contains(nextStateID)) {
        states += nextStateID -> State(nextStateID, Map(), endState = false)
      }

      // Move to the next state
      currentStateID = nextStateID
    }

    // Mark the last state of the keyword as an end state
    val finalState = states(currentStateID)
    states += currentStateID -> finalState.copy(endState = true, keyword = Some(keyword))  }

  states
}

defined function buildGraph

To run examples, a print function would be nice : 

In [3]:
def printGraph(states: Map[Int, State]): Unit = {
  states.foreach { case (id, state) =>
    val keywordStr = state.keyword match {
      case Some(kw) => s", Keyword = $kw"
      case None => ""
    }
    println(s"State $id: Successors = ${state.Successor}, End State = ${state.endState}$keywordStr")
  }
}

var keywords = List("hers", "she", "his")
printGraph(buildGraph(keywords))

State 0: Successors = Map(h -> 1, s -> 5), End State = false
State 5: Successors = Map(h -> 6), End State = false
State 1: Successors = Map(e -> 2, i -> 8), End State = false
State 6: Successors = Map(e -> 7), End State = false
State 9: Successors = Map(), End State = true, Keyword = his
State 2: Successors = Map(r -> 3), End State = false
State 7: Successors = Map(), End State = true, Keyword = she
State 3: Successors = Map(s -> 4), End State = false
State 8: Successors = Map(s -> 9), End State = false
State 4: Successors = Map(), End State = true, Keyword = hers


defined function printGraph
keywords: List[String] = List("hers", "she", "his")

Further, We need to implement a Failure function!

In [4]:

def computeFail( states: Map[Int, State]): Map[Int, Int] = {
  // Initialize fail function, setting all states to fail to root (state 0)
  var fail = Map[Int, Int]().withDefaultValue(0)
  val queue = scala.collection.mutable.Queue[Int]() // Using queue as suggested in the aho - corsick paper

  // Set failure links for root's direct successors
  for ((input, stateID) <- states(0).Successor) {
    fail += stateID -> 0 // direct successors of root point back to root!
    queue.enqueue(stateID)
  }


  // Breadth First Search : -----------------


  // BFS to compute failure links
  while (queue.nonEmpty) {
    val currentStateID = queue.dequeue()
    val currentState = states(currentStateID)

    // For each input and its successor state
    for ((input, successorID) <- currentState.Successor) {
      queue.enqueue(successorID)

      // Find the fail link for the current state
      var fallbackID = fail(currentStateID)
      while (fallbackID != 0 && !states(fallbackID).Successor.contains(input)) {
        fallbackID = fail(fallbackID)
      }


      // Update the fail link of the successor
      val fallbackSuccessorID = states(fallbackID).Successor.getOrElse(input, 0)
      fail += successorID -> fallbackSuccessorID

    }
  }

  fail
}

defined function computeFail

Now FSM

In [5]:
class FiniteStateMachine(SearchText: String, Keywords: List[String]) {

  private val states: Map[Int, State] = buildGraph(Keywords)
  private val text: String = SearchText
  private var keywords: List[String] = Keywords
  private val fails : Map[Int,Int] = computeFail(states)

  private var currentStateID: Int = 0 // Starting state is always 0

  def getCurrentStateID: Int = currentStateID // Simple getter for the current state ID

  /**
   * Perform pattern matching on the search text.
   * Returns a list of pairs with the index of the last character of a keyword and the keyword itself.
   */
  def PMM(): List[(Int, String)] = {
    var Output: List[(Int, String)] = List()
    if (text.nonEmpty && keywords.nonEmpty) {
      var charPos: Int = 0
      for (index <- 0 until text.length) {
        charPos += 1
        val gotoOutput: Int = goto(text.charAt(index).toString)

        if (gotoOutput == -1) {
          currentStateID = fail(currentStateID)
        } else {
          currentStateID = gotoOutput
          val currentStateOpt: Option[State] = states.get(currentStateID)
          currentStateOpt match {
            case Some(currentState) => if (currentState.endState) {
              Output = Output :+ (charPos, currentState.keyword.get)
            }
            case None => throw Exception(s"This State ID: $currentStateID does not exist!")
          }
        }
      }
    }
    Output
  }

  /**
   * GoTo function: takes an input and moves in the FSM.
   */
  private def goto(input: String): Int = {
    val currentStateOpt: Option[State] = states.get(currentStateID)
    currentStateOpt match {
      case Some(currentState) =>
        currentState.Successor.get(input) match {
          case Some(nextStateID) => nextStateID
          case None => if (currentStateID == 0) 0 else -1
        }
      case None => throw Exception(s"This State ID: $currentStateID does not exist!")
    }
  }

  /**
   * Fail function: returns the fail link for the current state.
   */
  def fail(stateID: Int): Int = fails(stateID)

}


defined class FiniteStateMachine

Run Example:

In [8]:
val fsm = new FiniteStateMachine("Scala is a great function and OOP language!", List("Scala", "functional", "OOP"))
val result = fsm.PMM()
println(result) 


List((5,Scala), (33,OOP))


fsm: FiniteStateMachine = ammonite.$sess.cmd5$Helper$FiniteStateMachine@40c1449d
result: List[(Int, String)] = List((5, "Scala"), (33, "OOP"))